In [2]:
#vv2289  - Vamshi

import csv
import pandas as pd
import sys
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from csv import reader

cf = SparkConf()
cf.set("spark.submit.deployMode", "client")
sc = SparkContext.getOrCreate(cf)
    
spark = SparkSession.builder.appName("Python Spark SQL basic example") \
        .config("spark.some.config.option", "some-value") \
        .getOrCreate()

csv_path = "/shared/dataWildlife/filtered.csv"

df =spark.read.option("escape", "\"").option('quote',"\"").option("multiline", True).csv(csv_path, inferSchema=True, header=True)

df_filtered = df.filter(df["animal_name"] != "")

df_filtered.show()



Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/25 21:15:44 INFO org.apache.spark.SparkEnv: Registering MapOutputTracker
24/04/25 21:15:44 INFO org.apache.spark.SparkEnv: Registering BlockManagerMaster
24/04/25 21:15:44 INFO org.apache.spark.SparkEnv: Registering BlockManagerMasterHeartbeat
24/04/25 21:15:44 INFO org.apache.spark.SparkEnv: Registering OutputCommitCoordinator


+--------------------+--------------------+--------------------+--------------------+--------------------+-------+--------+--------------------+--------------------+------------+---------------+-----------------+--------------------+------------+
|               title|         description|              seller|            location|             country|  price|currency|                  id|                 url|      domain|predicted_label|            match|         animal_name|product_type|
+--------------------+--------------------+--------------------+--------------------+--------------------+-------+--------+--------------------+--------------------+------------+---------------+-----------------+--------------------+------------+
|10 OSTRICH FEATHE...|10 OSTRICH Feathe...|       thefeatherguy|Flushing, New Yor...|       United States|  70.99|     USD|4585214d-b4d7-4c0...|http://picclick.c...|picclick.com|            1.0|          Ostrich|             Ostrich|     feather|
|1 PCS REAL 

In [3]:


df_filtered.printSchema()


unique_names_df = df_filtered.filter(df_filtered["animal_name"].isNotNull() & (df_filtered["animal_name"] != "")).select("animal_name").distinct()

unique_names_df.show(truncate=False)
unique_names = [row['animal_name'] for row in unique_names_df.collect()]


root
 |-- title: string (nullable = true)
 |-- description: string (nullable = true)
 |-- seller: string (nullable = true)
 |-- location: string (nullable = true)
 |-- country: string (nullable = true)
 |-- price: double (nullable = true)
 |-- currency: string (nullable = true)
 |-- id: string (nullable = true)
 |-- url: string (nullable = true)
 |-- domain: string (nullable = true)
 |-- predicted_label: double (nullable = true)
 |-- match: string (nullable = true)
 |-- animal_name: string (nullable = true)
 |-- product_type: string (nullable = true)



+-------------------------+
|animal_name              |
+-------------------------+
|Halibut                  |
|Sebastes fasciatus       |
|Silky shark              |
|Wolf                     |
|Hippoglossus hippoglossus|
|Calathus lundbladi       |
|Struthio camelus         |
|Caiman latirostris       |
|Redfish                  |
|Nile crocodile           |
|Ground beetle            |
|Sand tiger shark         |
|Carcharias taurus        |
|Carcharhinus falciformis |
|Great white shark        |
|Canis lupus              |
|Crocodylus niloticus     |
|Carcharhinus longimanus  |
|Ostrich                  |
+-------------------------+



Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.3/107.3 kB 1.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.6/311.6 kB 4.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.3/409.3 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 17.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.5 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-loc

In [7]:
!pip install --upgrade openai
!pip install boto3
import boto3
from botocore.exceptions import ClientError
import openai
import json

def check_species_status(species_name, api_key):
    client = openai.OpenAI(api_key=api_key)

    system_message = """
    You are a conservation expert with access to the latest IUCN Red List data. Determine whether the specified animal species is currently considered endangered, critically endangered, or least concern according to the IUCN Red List.
    """

    user_message = species_name

    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": system_message},
                {"role": "user", "content": user_message}
            ]
        )

        result = response.choices[0].message.content.strip().lower()
        is_endangered = 'endangered' in result or 'critically endangered' in result or 'threatened' in result
        return is_endangered
    except Exception as e:
        print(f"Error querying OpenAI about species status: {e}")
        return None

for name in unique_names:
    
    endangered = check_species_status(name, api_key)
    status = 'Endangered' if endangered else 'Not Endangered'
    if endangered is not None:
        print(f"{name}: {status}")
    else:
        print(f"{name}: Unable to determine status")

        
results = []
count=0;
api_key="xxxxxxxxx"
for name in unique_names:
    count+=1
    if(count>3): 
        break
    endangered = check_species_status(name, api_key)
    status = 'Endangered' if endangered else 'Not Endangered'
    if endangered is not None:
        results.append({'animal_name': name, 'status': status})

df1 = pd.DataFrame(results)

df1.to_csv('animal_info.csv', header=True, index=False)

print(df1.head())

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Halibut: Endangered
Sebastes fasciatus: Endangered
Silky shark: Not Endangered
Wolf: Not Endangered
Hippoglossus hippoglossus: Not Endangered
Calathus lundbladi: Not Endangered
Struthio camelus: Not Endangered
Caiman latirostris: Not Endangered
Redfish: Not Endangered
Nile crocodile: Not Endangered
Ground beetle: Endangered
Sand tiger shark: Not Endangered
Carcharias taurus: Not Endangered
Carcharhinus falciformis: Endangered
Great white shark: Not Endangered
Canis lupus: Endangered
Crocodylus niloticus: Not Endangered
Carcharhinus longimanus: Endangered
Ostrich: Not Endangered


FileNotFoundError: [Errno 2] No such file or directory: '/shared/dataWildlife/animal_info.csv'

In [14]:
!pip install --upgrade openai
!pip install boto3
import boto3
from botocore.exceptions import ClientError
import openai
import json
import pandas as pd
import os
from openai import OpenAI
import csv
import pandas as pd
import sys
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from csv import reader

cf = SparkConf()
cf.set("spark.submit.deployMode", "client")
sc = SparkContext.getOrCreate(cf)
    
spark = SparkSession.builder.appName("Python Spark SQL basic example") \
        .config("spark.some.config.option", "some-value") \
        .getOrCreate()

csv_path = "/shared/dataWildlife/filtered.csv"

df =spark.read.option("escape", "\"").option('quote',"\"").option("multiline", True).csv(csv_path, inferSchema=True, header=True)

df_filtered = df.filter(df["animal_name"] != "")

df_filtered.show()

def check_species_status(species_name, api_key):
    client = OpenAI(api_key=api_key)
    system_message = """
    You are a conservation expert with access to the latest IUCN Red List data. Determine whether the specified animal species is currently considered endangered, critically endangered, or least concern according to the IUCN Red List.
    """
    user_message = species_name
    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": system_message},
                {"role": "user", "content": user_message}
            ]
        )
        result = response.choices[0].message.content.strip().lower()
        is_endangered = 'endangered' in result or 'critically endangered' in result or 'threatened' in result
        return is_endangered
    except Exception as e:
        print(f"Error querying OpenAI about species status: {e}")
        return None

api_key="xxxxx"
if os.path.exists('animal_info.csv'):
    known_data = pd.read_csv('animal_info.csv')
else:
    known_data = pd.DataFrame(columns=['animal_name', 'status'])

new_data = []
all_data = known_data.copy()
for name in unique_names:
    if name not in known_data['animal_name'].values:
        endangered = check_species_status(name, api_key)
        status = 'Endangered' if endangered else 'Not Endangered'
        new_data.append({'animal_name': name, 'status': status})
        if endangered is not None:
            all_data = all_data.append({'animal_name': name, 'status': status}, ignore_index=True)

if new_data:
    new_df = pd.DataFrame(new_data)
    new_df.to_csv('animal_info.csv', mode='a', header=not os.path.exists('animal_info.csv'), index=False)

all_data.to_csv('output.csv', index=False)

print(all_data)


                  animal_name          status
0                     Halibut  Not Endangered
1          Sebastes fasciatus  Not Endangered
2                 Silky shark      Endangered
3                        Wolf  Not Endangered
4   Hippoglossus hippoglossus  Not Endangered
5          Calathus lundbladi  Not Endangered
6            Struthio camelus  Not Endangered
7          Caiman latirostris  Not Endangered
8                     Redfish  Not Endangered
9              Nile crocodile  Not Endangered
10              Ground beetle      Endangered
11           Sand tiger shark  Not Endangered
12          Carcharias taurus  Not Endangered
13   Carcharhinus falciformis      Endangered
14          Great white shark  Not Endangered
15                Canis lupus      Endangered
16       Crocodylus niloticus  Not Endangered
17    Carcharhinus longimanus      Endangered
18                    Ostrich  Not Endangered


In [ ]:
# -------------------------------------------------------------------

In [16]:
!pip install --upgrade openai
!pip install boto3
import boto3
from botocore.exceptions import ClientError
import openai
import json
import pandas as pd
import os
from openai import OpenAI
import csv
import pandas as pd
import sys
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from csv import reader

cf = SparkConf()
cf.set("spark.submit.deployMode", "client")
sc = SparkContext.getOrCreate(cf)
    
spark = SparkSession.builder.appName("Python Spark SQL basic example") \
        .config("spark.some.config.option", "some-value") \
        .getOrCreate()

csv_path = "/shared/dataWildlife/filtered.csv"

df =spark.read.option("escape", "\"").option('quote',"\"").option("multiline", True).csv(csv_path, inferSchema=True, header=True)

df_filtered = df.filter(df["animal_name"] != "")

unique_names_df = df_filtered.filter(df_filtered["animal_name"].isNotNull() & (df_filtered["animal_name"] != "")).select("animal_name").distinct()

unique_names = [row['animal_name'] for row in unique_names_df.collect()]


def check_species_status(species_name, api_key):
    client = OpenAI(api_key=api_key)
    system_message = """
    You are a conservation expert with access to the latest IUCN Red List data. Determine whether the specified animal species is currently considered endangered, critically endangered, or least concern according to the IUCN Red List.
    """
    user_message = species_name
    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": system_message},
                {"role": "user", "content": user_message}
            ]
        )
        result = response.choices[0].message.content.strip().lower()
        is_endangered = 'endangered' in result or 'critically endangered' in result or 'threatened' in result
        return is_endangered
    except Exception as e:
        print(f"Error querying OpenAI about species status: {e}")
        return None

api_key="xxxxxx"
if os.path.exists('animal_info.csv'):
    known_data = pd.read_csv('animal_info.csv')
else:
    known_data = pd.DataFrame(columns=['animal_name', 'status'])

new_data = []
all_data = known_data.copy()
for name in unique_names:
    if name not in known_data['animal_name'].values:
        endangered = check_species_status(name, api_key)
        status = 'Endangered' if endangered else 'Not Endangered'
        new_data.append({'animal_name': name, 'status': status})
        if endangered is not None:
            all_data = all_data.append({'animal_name': name, 'status': status}, ignore_index=True)

if new_data:
    new_df = pd.DataFrame(new_data)
    new_df.to_csv('animal_info.csv', mode='a', header=not os.path.exists('animal_info.csv'), index=False)

all_data.to_csv('output.csv', index=False)



Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
